In [ ]:
regex_reference = r'^Avis No ([\d-]+)$'
regex_publication_date = r'^publié le ([\d/]+)$'
regex_deadline = r'^Date limite de réponse : ([\d/]+)$'
regex_department = r'^Dépt. : ([\dAB]+)$'

In [ ]:
def fetch_page(original_reference):
    url = 'http://www.boamp.fr/avis/detail/{}'.format(original_reference)
    r = requests.get(url)
    assert r.status_code == 200
    print(len(r.text))
    
    data = {
        'original_reference': original_reference,
    }
    
    soup = BeautifulSoup(r.text, 'html.parser')
    
    title = soup.find_all('h1')[1].text
    data['title'] = title
    
    reference = soup.find('p', class_='avis-ref').text
    reference = re.match(regex_reference, reference).groups()[0]
    data['reference'] = reference

    publication_date = soup.find('p', class_='date-publishing').text
    publication_date = re.match(regex_publication_date, publication_date).groups()[0]
    publication_date = datetime.datetime.strptime(publication_date, '%d/%m/%Y').date()
    data['publication_date'] = publication_date

    deadline = soup.find('p', class_='date-response').text
    regex_result = re.match(regex_deadline, deadline)
    if regex_result:
        deadline = regex_result.groups()[0]
        deadline = datetime.datetime.strptime(deadline, '%d/%m/%Y').date()
        data['deadline'] = deadline

    department = soup.find('p', class_='avis-geo').find('span').text
    department = re.match(regex_department, department).groups()[0]
    data['department'] = department

    buyer = list(soup.find('p', class_='avis-geo'))[1].strip()
    data['buyer'] = buyer

    avis_type = soup.find('p', class_='type-avis').find_all('span')[0].text.strip()
    data['avis_type'] = avis_type

    avis_subtype = soup.find('p', class_='type-avis').find_all('span')[1].text.strip()
    data['avis_subtype'] = avis_subtype

    for title_inline in soup.find_all('div', class_='title-inline'):
        chunk_type = title_inline.find('h3').text.strip()
        content = title_inline.find('p').text.strip()

        if chunk_type == 'Références complètes\xa0:':
            data['reference_inline'] = content
        elif chunk_type == 'Liens vers avis initiaux :':
            data['avis_initiaux'] = content
        else:
            raise ValueError('Unknown chunk type "{}" with content "{}"'.format(chunk_type, content))
            
    details = str(soup.find('div', id='detail-avis-content'))
    data['details'] = details

    return data
